# Kaggle historical real-estate data analysis on U.S. mainland data

Data preprossing in Jupyterlab - cleanup low house counts, U.S. territories, fill missing values using **sklearn** `KNNImputer`, `LinearRegression`

# ---------------------------------------------------------------------------------------

### Import libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np

############### Import warnings + watermark ###############
from watermark import watermark
from warnings import filterwarnings
filterwarnings("ignore")
print(watermark())
print(watermark(iversions=True, globals_=globals()))

Last updated: 2023-03-24T14:31:29.629808-07:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.34.0

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 22.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

pandas: 1.3.5
numpy : 1.21.6



# ---------------------------------------------------------------------------------------

### Import original `csv` file from Kaggle

In [2]:
# Create pandas dataframe using pandas `read_csv`
# data_df = pd.read_csv("original_data.csv")

### Display head of original data

In [3]:
# Display head values
display(data_df.head(5))

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000.0,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,for_sale,80000.0,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,for_sale,67000.0,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,for_sale,145000.0,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,NaN
4,for_sale,65000.0,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,NaN


### Display tail of original data

In [4]:
# Display tail values
display(data_df.tail(5))

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
923154,for_sale,445000.0,1.0,2.0,0.99,"1008 King St, Chappaqua, NY, 10514",1008 King St,Chappaqua,New York,10514.0,1052.0,2011-05-09
923155,for_sale,418000.0,4.0,2.0,0.40,"3 Elmwood Dr, Monroe, NY, 10950",3 Elmwood Dr,Monroe,New York,10950.0,1650.0,2015-07-21
923156,for_sale,469000.0,4.0,2.0,0.18,"13 N Conger Ave, Congers, NY, 10920",13 N Conger Ave,Congers,New York,10920.0,2123.0,NaN
923157,for_sale,825000.0,5.0,5.0,0.79,"7 Miller Rd, Valley Cottage, NY, 10989",7 Miller Rd,Valley Cottage,New York,10989.0,3775.0,2010-06-02
923158,for_sale,88000.0,NaN,NaN,35.00,"D18 Half Moon Bay Dr, Croton On Hudson, NY, 10520",D18 Half Moon Bay Dr,Croton On Hudson,New York,10520.0,780.0,NaN


### Convert price to 2 decimal places

In [5]:
# Round `price` to 2 decimals
data_df["price"] = data_df["price"].round(2)

### Get original dataframe info

In [6]:
# Get original dataframe info
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923159 entries, 0 to 923158
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        923159 non-null  object 
 1   price         923088 non-null  float64
 2   bed           791456 non-null  float64
 3   bath          807967 non-null  float64
 4   acre_lot      649536 non-null  float64
 5   full_address  923159 non-null  object 
 6   street        921021 non-null  object 
 7   city          923085 non-null  object 
 8   state         923159 non-null  object 
 9   zip_code      922954 non-null  float64
 10  house_size    625316 non-null  float64
 11  sold_date     456396 non-null  object 
dtypes: float64(6), object(6)
memory usage: 84.5+ MB


### Find number of `rows` & `columns` for the original data, using `.shape` method

In [7]:
# Get original data shape to get number of rows and columns
data_df.shape

(923159, 12)

# ---------------------------------------------------------------------------------------

### Display `status` values for available properties

In [8]:
# Display unique `status` values -  Rows with all values present are: `status`, `price`, `full_address`, `state`
data_df["status"].unique()

array(['for_sale', 'ready_to_build'], dtype=object)

### Drop `ready_to_build` status value from analysis. Only keep `for_sale` status values.

In [9]:
# Drop all rows with `status = ready_to_build`
data_df = data_df[data_df["status"] != "ready_to_build"]

# Display unique `status` values
data_df["status"].unique()

array(['for_sale'], dtype=object)

### Count number of `NaN` values for each column for `for_sale` status

In [10]:
# Count the NaN values in each column
nan_counts = data_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed             131703
bath            113561
acre_lot        271992
full_address         0
street            2138
city                74
state                0
zip_code           205
house_size      297843
sold_date       465132
dtype: int64

### Display `for_sale` properties in all available locations

In [11]:
# Find all properties available in each state
display(data_df.groupby("state")["status"].size())

state
Connecticut        91632
Delaware            2092
Georgia               50
Maine              36650
Massachusetts     174488
New Hampshire      51394
New Jersey        232997
New York          211769
Pennsylvania       17066
Puerto Rico        24679
Rhode Island       29596
South Carolina        25
Tennessee             20
Vermont            46460
Virgin Islands      2573
Virginia              31
West Virginia          3
Wyoming                3
Name: status, dtype: int64

# ---------------------------------------------------------------------------------------

### Drop specific locations for analysis - U.S. territories, U.S mainland states with lower than 100 counts available

In [12]:
# Drop Puerto Rico & US Virgin Islands, states with low counts
states_to_drop = ["Virgin Islands", "Puerto Rico", "Georgia", "South Carolina", "Tennessee", "Virginia", "West Virginia", "Wyoming"]
filtered_df = data_df[~data_df["state"].isin(states_to_drop)]


# Display dataframe
display(filtered_df)

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
24231,for_sale,180000.0,2.0,1.0,0.34,"23 Moore St, Agawam, MA, 01001",23 Moore St,Agawam,Massachusetts,1001.0,676.0,NaN
24232,for_sale,25000.0,NaN,NaN,3.41,"Leonard St, Agawam, MA, 01001",Leonard St,Agawam,Massachusetts,1001.0,NaN,NaN
24233,for_sale,169900.0,2.0,2.0,NaN,"420 Main St Apt 42, Agawam, MA, 01001",420 Main St Apt 42,Agawam,Massachusetts,1001.0,892.0,2019-04-30
24234,for_sale,242000.0,2.0,2.0,NaN,"2A Mansion Woods Dr Unit 2A, Agawam, MA, 01001",2A Mansion Woods Dr Unit 2A,Agawam,Massachusetts,1001.0,1428.0,NaN
24235,for_sale,299950.0,2.0,2.0,NaN,"19 Castle Hill Rd Unit C, Agawam, MA, 01001",19 Castle Hill Rd Unit C,Agawam,Massachusetts,1001.0,1659.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
923154,for_sale,445000.0,1.0,2.0,0.99,"1008 King St, Chappaqua, NY, 10514",1008 King St,Chappaqua,New York,10514.0,1052.0,2011-05-09
923155,for_sale,418000.0,4.0,2.0,0.40,"3 Elmwood Dr, Monroe, NY, 10950",3 Elmwood Dr,Monroe,New York,10950.0,1650.0,2015-07-21
923156,for_sale,469000.0,4.0,2.0,0.18,"13 N Conger Ave, Congers, NY, 10920",13 N Conger Ave,Congers,New York,10920.0,2123.0,NaN
923157,for_sale,825000.0,5.0,5.0,0.79,"7 Miller Rd, Valley Cottage, NY, 10989",7 Miller Rd,Valley Cottage,New York,10989.0,3775.0,2010-06-02


### Re-iterate: Drop rows with less than 100 counts

In [13]:
# Drop states that have 100 and less properties available
state_counts = filtered_df.groupby("state")["status"].size()
state_counts = state_counts[state_counts >= 100]
selected_states = state_counts.index.tolist()
selected_df = filtered_df[filtered_df["state"].isin(selected_states)]

# Display dataframe
display(selected_df)

# Display unique states with available property count
display(state_counts)

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
24231,for_sale,180000.0,2.0,1.0,0.34,"23 Moore St, Agawam, MA, 01001",23 Moore St,Agawam,Massachusetts,1001.0,676.0,NaN
24232,for_sale,25000.0,NaN,NaN,3.41,"Leonard St, Agawam, MA, 01001",Leonard St,Agawam,Massachusetts,1001.0,NaN,NaN
24233,for_sale,169900.0,2.0,2.0,NaN,"420 Main St Apt 42, Agawam, MA, 01001",420 Main St Apt 42,Agawam,Massachusetts,1001.0,892.0,2019-04-30
24234,for_sale,242000.0,2.0,2.0,NaN,"2A Mansion Woods Dr Unit 2A, Agawam, MA, 01001",2A Mansion Woods Dr Unit 2A,Agawam,Massachusetts,1001.0,1428.0,NaN
24235,for_sale,299950.0,2.0,2.0,NaN,"19 Castle Hill Rd Unit C, Agawam, MA, 01001",19 Castle Hill Rd Unit C,Agawam,Massachusetts,1001.0,1659.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
923154,for_sale,445000.0,1.0,2.0,0.99,"1008 King St, Chappaqua, NY, 10514",1008 King St,Chappaqua,New York,10514.0,1052.0,2011-05-09
923155,for_sale,418000.0,4.0,2.0,0.40,"3 Elmwood Dr, Monroe, NY, 10950",3 Elmwood Dr,Monroe,New York,10950.0,1650.0,2015-07-21
923156,for_sale,469000.0,4.0,2.0,0.18,"13 N Conger Ave, Congers, NY, 10920",13 N Conger Ave,Congers,New York,10920.0,2123.0,NaN
923157,for_sale,825000.0,5.0,5.0,0.79,"7 Miller Rd, Valley Cottage, NY, 10989",7 Miller Rd,Valley Cottage,New York,10989.0,3775.0,2010-06-02


state
Connecticut       91632
Delaware           2092
Maine             36650
Massachusetts    174488
New Hampshire     51394
New Jersey       232997
New York         211769
Pennsylvania      17066
Rhode Island      29596
Vermont           46460
Name: status, dtype: int64

### Find number of `rows` & `columns` for the states selected dataframe

In [14]:
# Get data shape to get number of rows and columns
selected_df.shape

(894144, 12)

## List data for all available states in alphabetical order for data analysis

In [15]:
# Sort values by alphabetical state order
sorted_state_df = selected_df.sort_values("state", ascending=True)

display(sorted_state_df.head(5))
print("-------------------------------------------------------------------------------------------------")
display(sorted_state_df.tail(5))

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
394904,for_sale,400000.0,4.0,3.0,0.77,"15 Banks Rd, Simsbury, CT, 06070",15 Banks Rd,Simsbury,Connecticut,6070.0,2468.0,2021-10-18
414805,for_sale,77000.0,2.0,1.0,NaN,"5 Poplar Ln Unit A, Mansfield, CT, 06250",5 Poplar Ln Unit A,Mansfield,Connecticut,6250.0,810.0,NaN
414806,for_sale,150000.0,2.0,2.0,0.70,"2 Mission St, Sprague, CT, 06330",2 Mission St,Sprague,Connecticut,6330.0,3972.0,NaN
414807,for_sale,399900.0,4.0,3.0,1.78,"113 Laurel Hill Rd, Brooklyn, CT, 06234",113 Laurel Hill Rd,Brooklyn,Connecticut,6234.0,2904.0,1999-12-16
414808,for_sale,209000.0,5.0,3.0,0.29,"272 Prospect St, Windham, CT, 06226",272 Prospect St,Windham,Connecticut,6226.0,2848.0,1999-08-31


-------------------------------------------------------------------------------------------------


,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
348547,for_sale,152500.0,7.0,2.0,0.2,"6943 Main St, Readsboro, VT, 05350",6943 Main St,Readsboro,Vermont,5350.0,2912.0,2020-07-02
348546,for_sale,134900.0,1.0,1.0,NaN,"2185 Drake Rd Unit 4, Castleton, VT, 05735",2185 Drake Rd Unit 4,Castleton,Vermont,5735.0,780.0,NaN
277594,for_sale,79900.0,2.0,1.0,0.5,"496 Vermont Route 106, Weathersfield, VT, 05151",496 Vermont Route 106,Weathersfield,Vermont,5151.0,520.0,NaN
348559,for_sale,4850000.0,6.0,10.0,18.7,"135 Paddock Farm Ln, Dorset, VT, 05251",135 Paddock Farm Ln,Dorset,Vermont,5251.0,12126.0,NaN
370302,for_sale,599000.0,NaN,NaN,74.4,"2156 Blackmer Blvd, Stockbridge, VT, 05772",2156 Blackmer Blvd,Stockbridge,Vermont,5772.0,NaN,NaN


### List all unique mainland states for analysis

In [16]:
# Display filtered states & datatypes by count
unique_states = sorted_state_df["state"].unique()
display(unique_states)

array(['Connecticut', 'Delaware', 'Maine', 'Massachusetts',
       'New Hampshire', 'New Jersey', 'New York', 'Pennsylvania',
       'Rhode Island', 'Vermont'], dtype=object)

### Show counts of `NaN` values

In [17]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed             126429
bath            108468
acre_lot        266003
full_address         0
street            2093
city                23
state                0
zip_code            25
house_size      292240
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

## Fill empty values using `sklearn`

### Fill missing `zip_code` values

In [18]:
# Import KNNImputer function from the sklearn python package
from sklearn.impute import KNNImputer

# Create a KNN imputer object with k=5 and 'uniform' weighting
imputer = KNNImputer(n_neighbors=5, weights='uniform')

# Only select the 'zip_code' column with missing values
zip_df = sorted_state_df[['zip_code']]

# Impute missing 'zip_code' values using KNN imputation
zip_imputed = imputer.fit_transform(zip_df)

# Replace the original 'zip_code' column with the imputed values
sorted_state_df['zip_code'] = zip_imputed

In [19]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed             126429
bath            108468
acre_lot        266003
full_address         0
street            2093
city                23
state                0
zip_code             0
house_size      292240
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

### Fill missing `city` values using `zip_code`

In [20]:
# Group by zip_code and calculate mode of city for each group
mode_by_zip = sorted_state_df.groupby('zip_code')['city'].apply(lambda x: x.mode()[0])

# Create a boolean mask to identify the rows with missing city values
missing_city_mask = sorted_state_df['city'].isna()

# Fill in missing city values with corresponding mode for their zip_code
sorted_state_df.loc[missing_city_mask, 'city'] = sorted_state_df.loc[missing_city_mask, 'zip_code'].map(mode_by_zip)

### Show all missing counts

In [21]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed             126429
bath            108468
acre_lot        266003
full_address         0
street            2093
city                 0
state                0
zip_code             0
house_size      292240
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

## ML Linear Regression - fill missing values

### Fill `house_size` using `zip_code` & `price` as features for the Regression model

In [22]:
from sklearn.linear_model import LinearRegression

# Split the data into two sets: one with complete data for house_size and another with missing values
complete_data = sorted_state_df.dropna(subset=['house_size'])
missing_data = sorted_state_df.loc[sorted_state_df['house_size'].isna()]

# Define the features to be used in the regression model
features = ['zip_code', 'price']

# Fit the linear regression model using the complete data
reg = LinearRegression().fit(complete_data[features], complete_data['house_size'])

# Predict the missing house_size values using the fitted regression model
missing_data['house_size'] = reg.predict(missing_data[features])

# Replace the missing house_size values with the predicted values
sorted_state_df.loc[sorted_state_df['house_size'].isna(), 'house_size'] = missing_data['house_size']

In [23]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed             126429
bath            108468
acre_lot        266003
full_address         0
street            2093
city                 0
state                0
zip_code             0
house_size           0
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

### Fill `acre_lot` using `zip_code`, `price` & `house_size` as features for the Regression model

In [24]:
# Split the data into two sets: one with complete data for house_size and another with missing values
complete_data = sorted_state_df.dropna(subset=['acre_lot'])
missing_data = sorted_state_df.loc[sorted_state_df['acre_lot'].isna()]

# Define the features to be used in the regression model
features = ['zip_code', 'price', 'house_size']

# Fit the linear regression model using the complete data
reg = LinearRegression().fit(complete_data[features], complete_data['acre_lot'])

# Predict the missing house_size values using the fitted regression model
missing_data['acre_lot'] = reg.predict(missing_data[features])

# Replace the missing house_size values with the predicted values
sorted_state_df.loc[sorted_state_df['acre_lot'].isna(), 'acre_lot'] = missing_data['acre_lot']

In [25]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed             126429
bath            108468
acre_lot             0
full_address         0
street            2093
city                 0
state                0
zip_code             0
house_size           0
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

### Fill `bed` using `zip_code`, `price`, `house_size` & `acre_lot` as features for the Regression model

In [26]:
# Split the data into two sets: one with complete data for house_size and another with missing values
complete_data = sorted_state_df.dropna(subset=['bed'])
missing_data = sorted_state_df.loc[sorted_state_df['bed'].isna()]

# Define the features to be used in the regression model
features = ['zip_code', 'price', 'house_size', 'acre_lot']

# Fit the linear regression model using the complete data
reg = LinearRegression().fit(complete_data[features], complete_data['bed'])

# Predict the missing house_size values using the fitted regression model
missing_data['bed'] = reg.predict(missing_data[features])

# Replace the missing house_size values with the predicted values
sorted_state_df.loc[sorted_state_df['bed'].isna(), 'bed'] = missing_data['bed']

In [27]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed                  0
bath            108468
acre_lot             0
full_address         0
street            2093
city                 0
state                0
zip_code             0
house_size           0
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

### Fill `bath` using `zip_code`, `price`, `house_size`, `acre_lot` & `bed` as features for the Regression model

In [28]:
# Split the data into two sets: one with complete data for house_size and another with missing values
complete_data = sorted_state_df.dropna(subset=['bath'])
missing_data = sorted_state_df.loc[sorted_state_df['bath'].isna()]

# Define the features to be used in the regression model
features = ['zip_code', 'price', 'house_size', 'acre_lot', 'bed']

# Fit the linear regression model using the complete data
reg = LinearRegression().fit(complete_data[features], complete_data['bath'])

# Predict the missing house_size values using the fitted regression model
missing_data['bath'] = reg.predict(missing_data[features])

# Replace the missing house_size values with the predicted values
sorted_state_df.loc[sorted_state_df['bath'].isna(), 'bath'] = missing_data['bath']

In [29]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed                  0
bath                 0
acre_lot             0
full_address         0
street            2093
city                 0
state                0
zip_code             0
house_size           0
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

### Fill missing `street` values by extracting values from `full_address`

In [30]:
def extract_street_name(address):
    # Split the address by commas and take the first element
    street_address = address.split(',')[0].strip()
    # Split the street address by spaces and take the last two elements
    street_name = ' '.join(street_address.split()[-2:])
    return street_name

# Apply the function to fill in missing street names
sorted_state_df['street'] = sorted_state_df['street'].fillna(sorted_state_df['full_address'].apply(extract_street_name))

In [31]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed                  0
bath                 0
acre_lot             0
full_address         0
street               0
city                 0
state                0
zip_code             0
house_size           0
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

## Convert columns to appropriate data/ decimal types

### Round `bed` values to `integer`

In [32]:
# Round bed column to the nearest .0 value on the sorted dataframe
sorted_state_df['bed'] = np.rint(sorted_state_df['bed'])

### Round `bath` values to nearest .0 or .5 value

In [33]:
# Round `bath` column to the nearest .0 or .5 value
sorted_state_df['bath'] = sorted_state_df['bath'].apply(lambda x: round(x * 2) / 2)

### Convert `acre_lot` to 2 decimal places

In [34]:
# Convert acre_lot column to 2 decimal places on the dataframe
sorted_state_df['acre_lot'] = sorted_state_df['acre_lot'].round(2)

# Replace negative acre_lot values with "Unavailable" on the dataframe
sorted_state_df['acre_lot'] = sorted_state_df['acre_lot'].apply(lambda x: "Unavailable" if x < 0 else round(x, 2))

In [35]:
# Convert house_size column to 1 decimal places on the dataframe
sorted_state_df['house_size'] = sorted_state_df['house_size'].round(1)

### Convert `zip_code` values to show appropirate 5-digit zipcodes

In [36]:
# Convert zip_code to string and pad with leading zeros
sorted_state_df['zip_code'] = sorted_state_df['zip_code'].astype(int).astype(str).apply(lambda x: x.zfill(5))

In [37]:
# Display info
display(sorted_state_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 894144 entries, 394904 to 370302
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        894144 non-null  object 
 1   price         894144 non-null  float64
 2   bed           894144 non-null  float64
 3   bath          894144 non-null  float64
 4   acre_lot      894144 non-null  object 
 5   full_address  894144 non-null  object 
 6   street        894144 non-null  object 
 7   city          894144 non-null  object 
 8   state         894144 non-null  object 
 9   zip_code      894144 non-null  object 
 10  house_size    894144 non-null  float64
 11  sold_date     456283 non-null  object 
dtypes: float64(4), object(8)
memory usage: 88.7+ MB


None

In [38]:
# Count the NaN values in each column
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status               0
price                0
bed                  0
bath                 0
acre_lot             0
full_address         0
street               0
city                 0
state                0
zip_code             0
house_size           0
sold_date       437861
dtype: int64

# ---------------------------------------------------------------------------------------

### Convert all negative `acre_lot` values to 'Unavailable'

In [39]:
# Count rows where acre_lot is "Unavailable"
unavailable_count = (sorted_state_df['acre_lot'] == 'Unavailable').sum()

# Print the count
print(f"There are {unavailable_count} rows where `acre_lot` is 'Unavailable'.")

There are 161512 rows where `acre_lot` is 'Unavailable'.


# ---------------------------------------------------------------------------------------

### Convert all missing `sold_date` values to 'Unavailable'

In [40]:
# Replace missing sold_date values with "Unavailable"
sorted_state_df["sold_date"].fillna("Unavailable", inplace=True)

# Count the number of "Unavailable" sold_date values
unavailable_count = (sorted_state_df["sold_date"] == "Unavailable").sum()

# Print the count of "Unavailable" sold_date values
print(f"There are {unavailable_count} rows where `sold_date` is 'Unavailable'.")

There are 437861 rows where `sold_date` is 'Unavailable'.


In [41]:
# Count the NaN values in each column again
nan_counts = sorted_state_df.isna().sum()

# Print the NaN counts for each column
display(nan_counts)

status          0
price           0
bed             0
bath            0
acre_lot        0
full_address    0
street          0
city            0
state           0
zip_code        0
house_size      0
sold_date       0
dtype: int64

# ---------------------------------------------------------------------------------------

### Add a new column to `price_sq_ft` that shows a ratio of `price` / `house_size`

In [42]:
sorted_state_df['price_sq_ft'] = (sorted_state_df['price'] / sorted_state_df['house_size']).round(2)

# ---------------------------------------------------------------------------------------

### Use `bed` and `bath` equals to 1 as starting point

In [43]:
sorted_state_df = sorted_state_df.loc[(sorted_state_df['bed'] >= 1) & (sorted_state_df['bath'] >= 1)]

### Find datatypes of all the columns in `sorted_state_df`. Drop `status` column as everything is `for_sale` now

In [44]:
sorted_state_df = sorted_state_df.drop('status', axis=1)

In [45]:
display(sorted_state_df.dtypes)

price           float64
bed             float64
bath            float64
acre_lot         object
full_address     object
street           object
city             object
state            object
zip_code         object
house_size      float64
sold_date        object
price_sq_ft     float64
dtype: object

In [46]:
# Get final dataframe shape
display(sorted_state_df.shape)

(894115, 12)

# ---------------------------------------------------------------------------------------

## Save cleanup file `sorted_state_df` for further data analysis via ML

In [47]:
sorted_state_df.to_csv('sorted_state_df.csv', index=False)

## Create `X` & `y` columns for further training & testing

In [48]:
# Select columns for X and y for training & testing
X = sorted_state_df[['bed', 'bath', 'house_size', 'price_sq_ft']]
print(X.dtypes)
y = sorted_state_df['price']
print(y.dtypes)

bed            float64
bath           float64
house_size     float64
price_sq_ft    float64
dtype: object
float64


In [56]:
display(X.head())
print("----------------------------------------")
display(y.head())

,bed,bath,house_size,price_sq_ft
394904,4.0,3.0,2468.0,162.07
414805,2.0,1.0,810.0,95.06
414806,2.0,2.0,3972.0,37.76
414807,4.0,3.0,2904.0,137.71
414808,5.0,3.0,2848.0,73.38


----------------------------------------


394904    400000.0
414805     77000.0
414806    150000.0
414807    399900.0
414808    209000.0
Name: price, dtype: float64

# ---------------------------------------------------------------------------------------

## Split data for training & testing via **sklearn** package

In [50]:
# Import sklearn train_test_split library
from sklearn.model_selection import train_test_split

# Define params - use 20% as test data, and set random_state = 10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# ---------------------------------------------------------------------------------------

## Fit a ML model with training data & use testing data to evaluate the performance of the model on new, unseen data.

## `Regressor` predictions

#### Pick 5 regression models: `XGBoost`, `Linear Regression`, `Decision Tree`, `Random Forest`, and `Gradient Boosting`.
#### Pick scoring metrics: `Root Mean Square Error (RMSE)`, `R^2`, `Mean Absolute Error (MAE)`

In [51]:
# Import necessary libraries
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Define the regressor models to use
models = [
    ('XGBoost', XGBRegressor()),  # XGBoost regressor
    ('Linear Regression', LinearRegression()),  # Linear regression model
    ('Decision Tree', DecisionTreeRegressor()),  # Decision tree regressor
    ('Random Forest', RandomForestRegressor()),  # Random forest regressor
    ('Gradient Boosting', GradientBoostingRegressor())  # Gradient boosting regressor
]

# Evaluate the models using cross-validation and print the results
for name, model in models:
    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Compute the evaluation metrics on the test data
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Print the evaluation metrics for the model
    print(f"Model: {name}")
    print(f"  RMSE on test data: {rmse:.2f}")  # root mean squared error
    print(f"  R^2 on test data: {r2:.2f}")  # R-squared score
    print(f"  MAE on test data: {mae:.2f}")  # mean absolute error
    print("------")

# Find the winning model with the best evaluation metric
winning_model, winning_metric = min([(name, model) for name, model in models], key=lambda x: mean_squared_error(y_test, x[1].predict(X_test), squared=False))

# Compute the evaluation metrics for the winning model
y_pred = winning_metric.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Sort models by RMSE score in ascending order
sorted_models = sorted(models, key=lambda x: mean_squared_error(y_test, x[1].predict(X_test), squared=False))

# Display the winning model
print(f"The winning model is {winning_model} with the lowest RMSE score of {rmse:.2f} on the test data, indicating the best performance.")
print(f"The R^2 score of the winning model is {r2:.2f}, and the MAE score is {mae:.2f}.\n")

# Display the second best model
second_best_model = sorted_models[1][1]
y_pred = second_best_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"The second best model is {sorted_models[1][0]} with an RMSE score of {rmse:.2f} on the test data.")
print(f"The R^2 score of the second best model is {r2:.2f}, and the MAE score is {mae:.2f}.")

Model: XGBoost
  RMSE on test data: 51075.43
  R^2 on test data: 1.00
  MAE on test data: 19474.57
------
Model: Linear Regression
  RMSE on test data: 1129429.08
  R^2 on test data: 0.92
  MAE on test data: 400679.99
------
Model: Decision Tree
  RMSE on test data: 92450.34
  R^2 on test data: 1.00
  MAE on test data: 1103.26
------
Model: Random Forest
  RMSE on test data: 1328797.01
  R^2 on test data: 0.89
  MAE on test data: 6229.88
------
Model: Gradient Boosting
  RMSE on test data: 153769.29
  R^2 on test data: 1.00
  MAE on test data: 70008.41
------
The winning model is XGBoost with the lowest RMSE score of 51075.43 on the test data, indicating the best performance.
The R^2 score of the winning model is 1.00, and the MAE score is 19474.57.

The second best model is Decision Tree with an RMSE score of 92450.34 on the test data.
The R^2 score of the second best model is 1.00, and the MAE score is 1103.26.


# ---------------------------------------------------------------------------------------

## Conformal prediction

### Define a function to calculate the prediction intervals using the conformal prediction algorithm - `XGBoost`

In [52]:
def conformal_prediction(model, X_train, y_train, X_test, significance_level):
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    n_features = X_train.shape[1]
    alpha = significance_level / n_train

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Get the predicted values and residuals for the training data
    y_pred_train = model.predict(X_train)
    residuals_train = y_train - y_pred_train

    # Calculate the prediction intervals for the test data
    y_pred_test = model.predict(X_test)
    residuals_test = y_train - y_pred_test
    intervals = np.zeros((n_test, 2))
    for i in range(n_test):
        x_i = X_test[i, :]
        y_i = y_pred_test[i]
        r_i = residuals_test[i]
        sigma_i = np.sqrt(np.sum((residuals_train - r_i) ** 2) / (n_train - n_features - 1))
        quantile = np.quantile(np.abs(residuals_train - r_i) / sigma_i, 1 - alpha)
        interval = (y_i - quantile * sigma_i, y_i + quantile * sigma_i)
        intervals[i, :] = interval

    return intervals